In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
%matplotlib inline
sns.set()

print(os.listdir('../FinalWork/berlin-airbnb-data'))

df_reviews = pd.read_csv('../FinalWork/berlin-airbnb-data/reviews.csv')
df_neighbourhoods = pd.read_csv('../FinalWork/berlin-airbnb-data/neighbourhoods.csv')
df_listings = pd.read_csv('../FinalWork/berlin-airbnb-data/listings.csv')
df_calendar_summary = pd.read_csv('../FinalWork/berlin-airbnb-data/calendar_summary.csv')
df_reviews_summary = pd.read_csv('../FinalWork/berlin-airbnb-data/reviews_summary.csv')
df_listings_summary = pd.read_csv('../FinalWork/berlin-airbnb-data/listings_summary.csv')

df_listings_summary["host_since"].fillna(0, inplace=True)

df_listings_summary["host_since"] = df_listings_summary["host_since"].astype("str")

df_listings_summary["since"] = df_listings_summary["host_since"].apply(lambda x: x[:4])

new = []
new.append("since")

df_listings_summary.host_location.astype("str")

df_listings_summary["host_location"].fillna("f", inplace=True)

df_listings_summary["is_hostinBerlin"] = df_listings_summary["host_location"].apply(lambda x: "Berlin" in x)

new.append("is_hostinBerlin")

df_listings_summary["host_response_time"].fillna("unknown", inplace=True)

def get_one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

cols_list = ['host_response_time']
one_hot_df = get_one_hot(df_listings_summary[['host_response_time']], cols_list)

one_hot_host_response_time = pd.get_dummies(df_listings_summary[['host_response_time']])

df_listings_summary = pd.concat([df_listings_summary, one_hot_host_response_time], axis=1)


for i in one_hot_host_response_time.columns:
    new.append(i)

import warnings
warnings.filterwarnings("ignore")

df_listings_summary["host_response_rate"].fillna("0%", inplace=True)

df_listings_summary["response_rate"] = df_listings_summary["host_response_rate"].apply(lambda x: x.replace("%", ""))

new.append("response_rate")

df_listings_summary["host_is_superhost"].fillna("f", inplace=True)

new.append("host_is_superhost")

df_listings_summary["host_listings_count"].fillna(0.0, inplace=True)

new.append("host_listings_count")

df_listings_summary["host_has_profile_pic"].fillna("f", inplace=True)

new.append("host_has_profile_pic")

df_listings_summary["host_identity_verified"].fillna("f", inplace=True)

new.append("host_identity_verified")

one_hot_neighbourhood = pd.get_dummies(df_listings_summary[['neighbourhood_group_cleansed']])

df_listings_summary = pd.concat([df_listings_summary, one_hot_neighbourhood], axis=1)

for i in one_hot_neighbourhood.columns:
    new.append(i)

new.append("is_location_exact")

one_hot_room_type = pd.get_dummies(df_listings_summary[['room_type']])


df_listings_summary = pd.concat([df_listings_summary, one_hot_room_type], axis=1)

df_listings_summary["bathrooms"].fillna(1.0, inplace=True)

new.append("bathrooms")

new.append("accommodates")

df_listings_summary["bedrooms"].fillna(1.0, inplace=True)

new.append("bedrooms")

df_listings_summary["is_realbed"] = df_listings_summary["bed_type"].apply(lambda x: "Real Bed" in x)

new.append("is_realbed")

df_listings_summary["security_deposit"].fillna("$0.00", inplace=True)
df_listings_summary["cleaning_fee"].fillna("$0.00", inplace=True)
df_listings_summary["extra_people"].fillna("$0.00", inplace=True)

def remove_s(str_price):
    if "$" in str_price:
        str_price = str_price.replace("$", "")
    else:
        pass
    return str_price


for s_column in ["price", "security_deposit", "cleaning_fee", "extra_people"]:
    price_no_s = df_listings_summary[s_column].apply(remove_s)
    df_listings_summary[s_column] = price_no_s

def remove_c(str_price):
    if "," in str_price:
        str_price = str_price.replace(",", "")
    else:
        pass
    return str_price


for c_column in ["price", "security_deposit", "cleaning_fee", "extra_people"]:
    price_no_c = df_listings_summary[c_column].apply(remove_c)
    df_listings_summary[c_column] = price_no_c

df_listings_summary["price"] = df_listings_summary["price"].astype("float")

for i in ["security_deposit", "cleaning_fee", "extra_people"]:
    new.append(i)

for i in ["maximum_nights", "minimum_nights"]:
    new.append(i)

df_listings_summary["cleaning_fee"] = df_listings_summary["cleaning_fee"].astype("float")
df_listings_summary["security_deposit"] = df_listings_summary["security_deposit"].astype("float")
df_listings_summary["extra_people"] = df_listings_summary["extra_people"].astype("float")

for i in ["availability_30", "availability_60", "availability_90", "availability_365"]:
    new.append(i)

new.append("instant_bookable")

df_listings_summary["is_cancellation_flexible"] = df_listings_summary["cancellation_policy"].apply(lambda x: "flexible" in x)

new.append("is_cancellation_flexible")

new.append("calculated_host_listings_count")

df_listings_summary["reviews_per_month"].fillna("0.00", inplace=True)

new.append("reviews_per_month")

new.append("id")

new.append("price")

df_selected = df_listings_summary[new].set_index('id')

df_selected = df_selected[(df_selected.price 
                                 <= np.percentile(df_selected["price"],
                                99.5)) & (df_selected.price > 0)]

df_selected["since"] = df_selected["since"].astype("int")

df_selected["response_rate"] = df_selected["response_rate"].astype("int")

df_selected["host_is_superhost"] = df_selected["host_is_superhost"]=="t"

df_selected["host_has_profile_pic"] = df_selected["host_has_profile_pic"]=="t"

df_selected["host_identity_verified"] = df_selected["host_identity_verified"]=="t"

df_selected["is_location_exact"] = df_selected["is_location_exact"]=="t"

df_selected["instant_bookable"] = df_selected["instant_bookable"]=="t"

df_selected["reviews_per_month"] = df_selected["reviews_per_month"].astype("float")

for col in df_selected.select_dtypes("bool").columns:
    df_selected[col] = df_selected[col].astype("int")

df = df_selected

features = []
for col in df_selected.columns:
    if col != "price":
        features.append(col)

['reviews.csv', '.DS_Store', 'neighbourhoods.csv', 'listings.csv', 'calendar_summary.csv', 'reviews_summary.csv', '.ipynb_checkpoints', 'listings_summary.csv']


In [2]:
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
trans = PolynomialFeatures(degree=3)
PolyFeatures = trans.fit_transform(df[features])

In [5]:
X = PolyFeatures
y = df["price"]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
# регрессия
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.00


In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 [7 0 0 ... 0 0 0]]


In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       0.0
         8.0       0.00      0.00      0.00       1.0
         9.0       0.00      0.00      0.00       5.0
        10.0       0.00      0.00      0.00       7.0
        11.0       0.00      0.00      0.00       2.0
        12.0       0.00      0.00      0.00       8.0
        13.0       0.00      0.00      0.00      11.0
        14.0       0.00      0.00      0.00      16.0
        15.0       0.00      0.00      0.00      58.0
        16.0       0.00      0.00      0.00      23.0
        17.0       0.00      0.00      0.00      34.0
        18.0       0.00      0.00      0.00      48.0
        19.0       0.00      0.00      0.00      39.0
        20.0       0.00      0.00      0.00     198.0
        21.0       0.00      0.00      0.00      28.0
        22.0       0.00      0.00      0.00      75.0
        23.0       0.00      0.00      0.00      54.0
        24.0       0.00    

In [12]:
RMSE_reg = np.sqrt(mean_squared_error(y_test, y_pred))
mae_reg = metrics.mean_absolute_error(y_test, y_pred)
print(f"RMSE: {round(RMSE_reg, 4)}")
print("MAE:",mae_reg)

RMSE: 71.3844
MAE: 56.953215505718106


In [13]:
from sklearn.svm import SVR

In [ ]:
#SVR
regressor = SVR(kernel='rbf')
regressor.fit(X_train, y_train)

In [ ]:
y_pred2 = regressor.predict(X_test)
RMSE_svr = np.sqrt(mean_squared_error(y_test, y_pred2))
mae_svr = metrics.mean_absolute_error(y_test, y_pred2)
print(f"RMSE: {round(RMSE_svr, 4)}")
print("MAE:",mae_svr)

In [ ]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из сотни деревьев
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')

model.fit(X_train, y_train)

In [ ]:
# Действующая классификация
rf_predictions = model.predict(X_test)
# Вероятности для каждого класса
rf_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test, rf_predictions))
mae = metrics.mean_absolute_error(y_test, rf_predictions)
print(f"RMSE: {round(RMSE, 4)}")
print("MAE:",mae)

In [6]:
import xgboost as xgb

In [ ]:
xgb_clf = xgb.XGBRegressor()
parameters = {'n_estimators': [120, 100, 140], 'max_depth':[3,5,7,9]}
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [9]:
xgb_clf = xgb.XGBRegressor(n_estimators=140, max_depth=5)
xgb_clf.fit(X_train, y_train)
y_test_pred = xgb_clf.predict(X_test)
print("R^2 score: {0: .4f}".format(r2_score(y_test, y_test_pred)))
print("RMSE: {0: .4f}".format(np.sqrt(mean_squared_error(y_test, y_test_pred))))
mae = metrics.mean_absolute_error(y_test, y_test_pred)
print("MAE:",mae)

[01:12:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R^2 score:  0.5344
RMSE:  29.3639
MAE: 17.893672883625673


In [10]:
metrics = pd.read_excel('./metrics.xlsx')
metrics.head()

,Unnamed: 0,Feature Selection Data,Unnamed: 2,Polynominal Data,Unnamed: 4,Full Dataset,Unnamed: 6,Polynominal Full Dataset,Unnamed: 8
0,NaN,RMSE,MAE,RMSE,MAE,RMSE,MAE,RMSE,MAE
1,LogisticRegression,35.1951,20.4705,79.0,46.5611,36.954,21.7375,71.3844,56.9532
2,SVR,32.5832,18.3961,35.4302,21.0012,35.1407,20.2288,NaN,NaN
3,RandomForest,35.5791,19.7946,35.8057,19.6206,34.5738,20.4075,NaN,NaN
4,xgboost,26.5518,15.2361,27.0310,15.7016,28.8621,17.8052,NaN,NaN
